# Graded Exercise #3

[EuroSAT](https://zenodo.org/records/7711810#.ZAm3k-zMKEA) is a land use and land cover classification dataset. The dataset is based on Sentinel-2 satellite imagery covering 13 spectral bands and consists of 10 Land Use and Land Cover (LULC) classes with a total of 27,000 labeled and geo-referenced images. 

Using the following code, you can create a DataFrame that includes columns detailing the center locations of the images and their corresponding Land Use and Land Cover (LULC) classes.

Our task: Perform **supervised Machine Learning (ML)** on this dataset. Later we will use **Dask** in your implementation. We will follow the following key steps:
- Create relevant features and analyse them using some visualizations and statistical tools. You can start with features representing the mean and range of spectral bands in these images. You are free to explore more relevant features.
- Split the dataset into training, validation, and test sets.
- Choose an appropriate ML algorithm.
- Train and assess the model's performance.
- Adjust the model's hyperparameters to optimize its performance.

**Points of discussion**:
- Can you explain the criteria and rationale behind the features you created? What other features you would select from these images in addition to the mean and range? --> See section 2 of Assignement_Without_Dask
- Why is it important to have separate sets for training, validation, and testing? Which split did you consider and why? --> See section 3 of Assignement_Without_Dask   
- What factors influenced your choice of a specific machine learning algorithm? --> See section 4 of Assignement_Without_Dask    
- How did hyperparameter tuning impact the model's performance, and what were the final hyperparameter settings? --> See section 5 of Assignement_Dask
- What is the impact of using DASK to solve this problem? What is the impact of changing DASK parameters like chunk size? You may consider checking CPU, memory usage, processing time, ... --> See section 6 of Assignement_Dask

## 1. Creating the work space
Import all the relevant folders and include the file path to the where the imagery data are. 

In [ ]:
import numpy as np
import pandas as pd
import rasterio
import rasterio.features
import rasterio.warp
import geojson
import matplotlib.pyplot as plt
from scipy import stats

In [ ]:
import os
from os.path import isfile, join

In [ ]:
directory_path = 'EuroSAT_MS/'

## 2. Creating target variable data frame and relevant features
The goal of this excersize is to work on our understanding of machine learning concepts and try to put it into practise. Using the imagery provided by the professor we will create a dataframe with the lat, long, respective land use classification and relevant features of centroid point for each image. Land use classification is our target variable and the readings from the bands, as well as the derived features, will be used to create a model that would accurate identify land use classification in a new set up imager. 

While the band readings are nice, we do not need to let the machine do all the work. There are several indexes that have been proven to help identify land use classification. The following indexes will be created as additional features: 

1.	Normalized Difference Vegetation Index (NDVI)- this versatile index is used in agriculture, natural hazards such as landslides, land use/land cover change detection, environmental monitoring, water resources etc. to name a few. NDVI provides valuable information in wide range of applications making it an important feature to be studied.
NDVI = (B8-B4) / (B8+B4).

2.	SAVI- Soil Adjusted Vegetation Index (SAVI) is used to correct Normalized Difference Vegetation Index (NDVI) for the influence of soil brightness in areas where vegetative cover is low. The higher the NDVI values (the same stands for SAVI) the denser (and healthier) the vegetation. But NDVI start saturating after the value of 0.7, while SAVI at this point is only 0.3. This means that SAVI can be better used in dense vegetation because it saturates less fast. 
For Sentinel-2 the formula is:
(B08 - B04) / (B08 + B04 + L) * (1.0 + L); L = 0.428
where: L is a soil brightness correction factor ranging from 0 to 1
L = 1 low vegetation cover, L = 0 high vegetation cover, L = 0.5 intermediate vegetation cover.
https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/savi/

3.	Normalised difference water index (NDWI)- is used to highlight open water features in a satellite image, allowing a water body to “stand out” against the soil and vegetation. The downside of the index is that it is sensitive to built structures, which can lead to overestimation of water bodies.
For Sentinel 2 data:
NDWI= (Band 3 – Band 8)/(Band 3 + Band 8)
NDWI: Index Formula, Value Range, And Uses In Agriculture (eos.com)

The funtion below given in this section gets the NDVI, NDWI and SAVI for the centroid point of each image.


In [ ]:
# Function to calculate NDVI, NDWI, and SAVI based on band values
def calculate_indices(bands):
    band4 = bands[3]  # Red
    band3 = bands[2]  # Green
    band8 = bands[7]  # NIR

    # Define the formula constants for SAVI
    L = 0.428
    # calculate the indices
    ndvi = (band8 - band4) / (band8 + band4)
    ndwi = (band3 - band8) / (band3 + band8)
    savi = (band8 - band4) / (band8 + band4 + L) * (1.0 + L)
    return ndvi, ndwi, savi

Right now we need to collect and create variables from the imagery to build out model. We wanted to include the max, median, range, and mode of all the band readings to have a broad understanding of what is going on at each observation point (centre coordinate). The funtion below reads all the tiff data and gets the data of the the centroid points of each image. The correspondentlat, lon, ndvi, ndwi, savi, max, min, range, mean and all band value are recorded for the centroid point of each image.

In [ ]:
# Function to read all tiff files into dataframe together with central coordinates and relevant features
def read_data (directory_path):
    # Build a datframe
    data = pd.DataFrame(columns=["Lat", "Lon", "Lulc_class", "mean_val", "range_val", "median_val", "min_val", "max_val", "ndvi", "ndwi", "savi",
                             "band1", "band2", "band3", "band4", "band5", "band6", "band7", "band8", "band9", "band10", "band11", "band12"])
    # Loop through the subfolders
    for subdir in os.listdir(directory_path):
        subfolder_path = os.path.join(directory_path, subdir)
        if os.path.isdir(subfolder_path):
        # Get a list of all TIF files in the subfolder
            tif_files = [file for file in os.listdir(subfolder_path) if file.endswith('.tif')]

            for tif_file in tif_files:
                tif_path = os.path.join(subfolder_path, tif_file)
                # Read the raster file using rasterio
                with rasterio.open(tif_path) as src:
                    # Read and stack all bands into a single array
                    bands = src.read()
                    # Transfer the uint16 to int16
                    bands = bands.astype(np.int16)

                    # Get the center coordinates and  pixel location
                    lon, lat = src.xy(src.width // 2, src.height // 2)
                    row, col = src.index(lon, lat)
                    bands = bands[:, row, col]
                    # Extract LULC class from the subfolder name
                    lulc_class = subdir

                    # Calculate NDVI, NDWI, and SAVI based on band values
                    ndvi, ndwi, savi = calculate_indices(bands)

                    # Calculate the requested statistics
                    mean_val = np.mean(bands)
                    range_val = np.ptp(bands)
                    median_val = np.median(bands)
                    min_val = np.min(bands)
                    max_val = np.max(bands)

                    # Append the data to the DataFrame
                    new_row = [lat, lon, lulc_class, mean_val, range_val,  median_val, min_val, max_val, ndvi, ndwi, savi, 
                            bands[0], bands[1], bands[2], bands[3], bands[4], bands[5], bands[6], bands[7],bands[8], bands[9], bands[10], bands[11]]
                    data = pd.concat([data, pd.DataFrame([new_row], columns=data.columns)], ignore_index=True)

    return data
    

In [ ]:
# Read the data from the images
data = read_data(directory_path)

## 3. Split the data in training, test, and validation
For our work it was important to have separate sets for training, validation, and testing. The primary reason was to reduce over fitting. The separation between these sets ensures that the model can perform well in real-world scenarios. Using the same data for all the three purposes can lead to over fitting as the model will simply memorize the data rather than making meaningful results. We chose to do K Fold cross validations for our work as it is less biased than the simple train/test/valid split. This method of spliting data will ideally not result in overfitting, while still being relativelly simple to implement. 

In [ ]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
data

In [ ]:
# Get the input and output data
#X = data[['mean_val', 'range_val', 'median_val', 'min_val', 'max_val', 'ndvi', 'ndwi', 'savi']]
X = data[['band1', 'band2', 'band3', 'band4', 'band5', 'band6', 'band7', 'band8', 'band9', 'band10', 'band11', 'band12']]
#X = data.drop(['Lulc_class'], axis=1)
y = data['Lulc_class']
# Create the k-fold object
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)

## 4. Developing our model
Random forest is a well-known machine learning model, commonly used for classification tasks. In recent studies random forest model was found to out preform artificial neural networks with the same task of land use classification [1] When working with Sentinel-2 specifically, as we are here, random forest was found to be the stand out model for land use/land cover classification [2]. For these reasons we chose random forest as our model.

[1] https://www.frontiersin.org/articles/10.3389/frai.2022.964279/full#:~:text=We%20classified%20land%20use%20and,conducted%20by%20Tan%20et%20al.

[2] Ge, G., Shi, Z., Zhu, Y., Yang, X., & Hao, Y. (2020). Land use/cover classification in an arid desert-oasis mosaic landscape of China using remote sensed imagery: Performance assessment of four machine learning algorithms. Global Ecology and Conservation, 22, e00971. https://www.sciencedirect.com/science/article/pii/S2351989420300202

In [ ]:
# Create the random forest classification model
model = RandomForestClassifier(max_depth=5, random_state=95)
# A list for accuaracy
acc_score = []
 
# Get the training and testing data using the k-fold object
for train_index , test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index], X.iloc[test_index]
    y_train , y_test = y.iloc[train_index], y.iloc[test_index]
    # Train the model
    model.fit(X_train,y_train)
    pred_values = model.predict(X_test)
    # Calculate the accuracy of the model
    acc = accuracy_score(pred_values,y_test)
    acc_score.append(acc)
     
avg_acc_score = sum(acc_score)/k

## 5.Hyperparameter Optimization
- Hyperparameter optimization is the process of finding the configuration of hyperparameters that results in the best performance. Hyperparameters are the variables that control the training process and the topology of an ML model. 
- The hyperparameters of a random forest model are the number of trees in the forest, the number of features to consider when looking for the best split, the maximum depth of the tree, the minimum number of samples required to split an internal node, the minimum number of samples required to be at a leaf node, and the number of features to consider when looking for the best split. 
- We used the RandomizedSearchCV and GridSearchCV function to find the best hyperparameters for our model.

In [ ]:
from time import time
from scipy.stats import randint as sp_randint

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

Randomized Search:
- A big max_depth could produce over-fitted trees whule a small max_depth could lead to under-fitting. Considering about our feature number, the range of max_depth is set from 1 to 50.
- Our model had around 10 features. There for the range of max_features was set to 1 to 10.
- Min_sample_split is the minimum number of samples required to split an internal node. The range of min_sample_split was set to 2 to 10 as bigger number could lead to under-fitting and smaller number could lead to over-fitting.


In [ ]:
# specify parameters and distributions to sample from
clf = RandomForestClassifier()

# define the parameter space that will be searched over
param_dist = {"max_depth": [1,2,3,4,5,10,15,20,50,None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(2, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# iterate over the training dataset multiple times
n_iter_search = 10

# run randomized search
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=10)

start = time()
random_search.fit(X_train, y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))

random_search.best_params_

In [ ]:
# Run the model with best parameters
best_random = random_search.best_estimator_
best_random.fit(X_train, y_train)
pred_labels = best_random.predict(X_test)
pa_mp = accuracy_score(y_test, pred_labels, normalize=False)
print("Classification accuracy of RF (Random Search) is", pa_mp/len(y_test))

Grid Search:
As grid search performs exhaustive search over specified parameter values for an estimator, we used the less range of values for max_depth, max_features and min_sample_split as we did for randomized search.
<p><span style="color:red">(The running process could be very long, please pay attention):</span></p>


- A big max_depth could produce over-fitted trees whule a small max_depth could lead to under-fitting. Considering about our feature number, the range of max_depth is set from 1 to 50.
- Our model had around 10 features. There for the range of max_features was set to 1 to 10.
- Min_sample_split is the minimum number of samples required to split an internal node. The range of min_sample_split was set to 2 to 10 as bigger number could lead to under-fitting and smaller number could lead to over-fitting.


In [ ]:
# define the parameter space that will be searched over
param_grid = {"max_depth": [1,5,20,50,None],
              "max_features": [3,5,10],
              "min_samples_split": [2,5],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run grid search
grid_search = GridSearchCV(clf, param_grid=param_grid, cv=10)
start = time()
grid_search.fit(X_train, y_train)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))

grid_search.best_params_

In [ ]:
# Run the model with best parameters
best_grid = grid_search.best_estimator_
best_grid.fit(X_train, X_test)
pred_labels = best_grid.predict(y_train)
pa_mp = accuracy_score(y_test, pred_labels, normalize=False)
print("Classification accuracy of RF (Grid Search) is", pa_mp/len(y_test))